In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer
from skorch.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline
import pickle
def identity_tokenizer(text):
    return text

In [2]:
with open("x_train.txt", "rb") as fp:   # Unpickling
    x_train = pickle.load(fp)
with open("y_train.txt", "rb") as fp:   # Unpickling
    y_train = pickle.load(fp)
with open("x_test.txt", "rb") as fp:   # Unpickling
    x_test = pickle.load(fp)
with open("y_test.txt", "rb") as fp:   # Unpickling
    y_test = pickle.load(fp)

### Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()


def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)

pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("log_model", log_model)])


pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)
rmse = mean_squared_error(y_test, y_pred, squared = False)
print('RMSE:', rmse)

D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
D:\Documents\anaconda3\envs\DataSciEnv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


ValueError: Unknown label type: 'continuous'

### Multiple Regression

In [4]:
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()


def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)

pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True,token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("lin_model", lin_model)])


pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)
rmse = mean_squared_error(y_test, y_pred, squared = False)
print('RMSE:', rmse)

RMSE: 2.0633279672117206


### Ridge Regression

In [5]:
from sklearn.linear_model import Ridge
ridge_model = Ridge()


def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)

pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True,token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("ridge_model", ridge_model)])


pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)
rmse = mean_squared_error(y_test, y_pred, squared = False)
print('RMSE:', rmse)

RMSE: 1.9896362


In [23]:
rangelist = np.logspace(-5, 2, 50)
print(rangelist)

[1.00000000e-05 1.38949549e-05 1.93069773e-05 2.68269580e-05
 3.72759372e-05 5.17947468e-05 7.19685673e-05 1.00000000e-04
 1.38949549e-04 1.93069773e-04 2.68269580e-04 3.72759372e-04
 5.17947468e-04 7.19685673e-04 1.00000000e-03 1.38949549e-03
 1.93069773e-03 2.68269580e-03 3.72759372e-03 5.17947468e-03
 7.19685673e-03 1.00000000e-02 1.38949549e-02 1.93069773e-02
 2.68269580e-02 3.72759372e-02 5.17947468e-02 7.19685673e-02
 1.00000000e-01 1.38949549e-01 1.93069773e-01 2.68269580e-01
 3.72759372e-01 5.17947468e-01 7.19685673e-01 1.00000000e+00
 1.38949549e+00 1.93069773e+00 2.68269580e+00 3.72759372e+00
 5.17947468e+00 7.19685673e+00 1.00000000e+01 1.38949549e+01
 1.93069773e+01 2.68269580e+01 3.72759372e+01 5.17947468e+01
 7.19685673e+01 1.00000000e+02]


In [24]:
rmselist = []
for r in rangelist:
    ridge_model = Ridge(alpha = r)


    def typechange(x):
        return x.astype(dtype = np.float32)
    typetransform = FunctionTransformer(typechange)

    pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True,token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("ridge_model", ridge_model)])


    pipe.fit(X=x_train, y=y_train)
    y_pred = pipe.predict(x_test)
    rmse = mean_squared_error(y_test, y_pred, squared = False)
    print(r,'RMSE:', rmse)
    rmselist.append(rmse)
    
min_index = rmselist.index((min(rmselist)))
print("Optimum parameters:", rangelist[min_index],":",min(rmselist))

1e-05 RMSE: 2.0167823
1.3894954943731388e-05 RMSE: 2.0132132
1.9306977288832496e-05 RMSE: 2.0154734
2.6826957952797274e-05 RMSE: 2.0137775
3.727593720314938e-05 RMSE: 2.0124006
5.1794746792312125e-05 RMSE: 2.0130188
7.196856730011514e-05 RMSE: 2.0124938
0.0001 RMSE: 2.0127099
0.00013894954943731373 RMSE: 2.0120583
0.00019306977288832496 RMSE: 2.012334
0.00026826957952797245 RMSE: 2.010891
0.0003727593720314938 RMSE: 2.0105393
0.0005179474679231213 RMSE: 2.0092287
0.0007196856730011514 RMSE: 2.0082896
0.001 RMSE: 2.006572
0.0013894954943731374 RMSE: 2.005066
0.0019306977288832496 RMSE: 2.003894
0.0026826957952797246 RMSE: 2.0026326
0.003727593720314938 RMSE: 2.0013375
0.005179474679231208 RMSE: 2.0005932
0.007196856730011514 RMSE: 1.9993536
0.01 RMSE: 1.9985099
0.013894954943731374 RMSE: 1.9977165
0.019306977288832496 RMSE: 1.9968549
0.026826957952797246 RMSE: 1.9960574
0.03727593720314938 RMSE: 1.9952542
0.05179474679231207 RMSE: 1.9945084
0.07196856730011514 RMSE: 1.9937649
0.1 RMSE: 

### Lasso

In [7]:
from sklearn.linear_model import Lasso
lasso_model = Lasso()


def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)

pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True,token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("lasso_model", lasso_model)])


pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)
rmse = mean_squared_error(y_test, y_pred, squared = False)
print('RMSE:', rmse)

RMSE: 2.0521414


### Elastic Net

In [15]:
from sklearn.linear_model import ElasticNet
ElasticNet_model = ElasticNet()


def typechange(x):
    return x.astype(dtype = np.float32)
typetransform = FunctionTransformer(typechange)

pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(tokenizer=identity_tokenizer, input="array", lowercase=False, norm="l2", max_features=None, sublinear_tf=True, stop_words="english",token_pattern=r'[^\s]+')), ("typetransform", typetransform), ("ElasticNet_model", ElasticNet_model)])


pipe.fit(X=x_train, y=y_train)
y_pred = pipe.predict(x_test)
rmse = mean_squared_error(y_test, y_pred, squared = False)
print('RMSE:', rmse)

RMSE: 2.0521414
